# Heart disease training model

In [1]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split

Importar el csv:

In [2]:
class HeartDataset(Dataset):
    def __init__(self, file, transform=None, target_transform=None):
        data = pd.read_csv(file)
        self.X = torch.tensor(data.iloc[:, :-1].values, dtype=torch.float32)
        self.y = torch.tensor(data.iloc[:, -1].values, dtype=torch.float32).unsqueeze(1)

        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        features = self.X[index]
        label = self.y[index]

        if self.transform:
            features = self.transform(features)
        if self.target_transform:
            label = self.target_transform(label)

        return features, label
    

dataset = HeartDataset(file='data/heart.csv')

Dividir les dades en train (80%) i test (20%) de forma aleatòria fent servir ``random_split``

In [3]:
dataset_len = len(dataset)
train_size = int(dataset_len * 0.8) # Les dades d'entrenament son el 80% del dataset
test_size = dataset_len - train_size # Les dades de test son el 20% restant
train_data, test_data = random_split(dataset, [train_size, test_size])

DataLoader:

In [4]:
bs = 32
train_dataloader = DataLoader(train_data, batch_size=bs, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=bs, shuffle=True)

Dispositiu que utilitzarem (GPU):

In [5]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


/home/iticbcn/Escritorio/UA/ex-basics/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Definim el model neuronal:

In [6]:
class HeartModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(13,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,1)
        )
    
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = HeartModel().to(device)
print(model)

HeartModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=13, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)


Funcions d'entrenament i de test

In [7]:
learning_rate = 0.003

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * bs + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += ((torch.sigmoid(pred) > 0.5) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Entrenar el model

In [8]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 4.340566  [   32/  820]
Test Error: 
 Accuracy: 59.0%, Avg loss: 0.715720 

Epoch 2
-------------------------------
loss: 0.747570  [   32/  820]
Test Error: 
 Accuracy: 72.7%, Avg loss: 0.556916 

Epoch 3
-------------------------------
loss: 0.422900  [   32/  820]
Test Error: 
 Accuracy: 68.3%, Avg loss: 0.542659 

Epoch 4
-------------------------------
loss: 0.545413  [   32/  820]
Test Error: 
 Accuracy: 75.1%, Avg loss: 0.492983 

Epoch 5
-------------------------------
loss: 0.523508  [   32/  820]
Test Error: 
 Accuracy: 73.7%, Avg loss: 0.515466 

Epoch 6
-------------------------------
loss: 0.477810  [   32/  820]
Test Error: 
 Accuracy: 70.7%, Avg loss: 0.548085 

Epoch 7
-------------------------------
loss: 0.553605  [   32/  820]
Test Error: 
 Accuracy: 77.1%, Avg loss: 0.504480 

Epoch 8
-------------------------------
loss: 0.547155  [   32/  820]
Test Error: 
 Accuracy: 77.1%, Avg loss: 0.474979 

Epoch 9
----------------